In [1]:
#!/usr/bin/env python

In [2]:
import os
import sys
if os.path.exists('/home/chieh/code/wPlotLib'):
	sys.path.insert(0,'/home/chieh/code/wPlotLib')
if os.path.exists('/home/chieh/code/wuML'):
	sys.path.insert(0,'/home/chieh/code/wuML')

In [3]:
import wuml
import numpy as np
import torch
import wplotlib
from wuml import HSIC

In [4]:
data = wuml.wData(xpath='../../data/wine.csv', ypath='../../data/wine_label.csv', randomly_shuffle_batch=True,
					batch_size=20, label_type='discrete', encode_discrete_label_to_one_hot=True )

Remember that you are trying to "minimize" this objective

In [5]:
def costFunction(x, y, ŷ, ind):
	H = HSIC(ŷ, y, X_kernel='linear', Y_kernel='linear', sigma_type='mpd' )
	# compare the HSIC value against numpy library
	#y = y.detach().cpu().numpy()
	#ŷ = ŷ.detach().cpu().numpy()
	#H2 = HSIC(ŷ, y, X_kernel='linear', Y_kernel='linear', sigma_type='mpd' )
	#import pdb; pdb.set_trace()
	return -H  

----------------------------------------------<br>
Create basic network and train

In [6]:
bNet = wuml.basicNetwork(costFunction, data, networkStructure=[(30,'relu'),('bn', True), (50,'relu'),('bn', True),(3,'none')], max_epoch=500, learning_rate=0.001)
bNet.train(print_status=True)
Ŷ = bNet(data, output_type='ndarray')		#Takes Numpy array or Tensor as input and outputs a Tensor

Network Info:
	Learning rate: 0.001
	Max number of epochs: 500
	Cost Function: costFunction
	Train Loop Callback: None
	Cuda Available: True
	Network Structure
		Linear(in_features=13, out_features=30, bias=True) , relu
		BatchNorm1d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True) 
		Linear(in_features=30, out_features=50, bias=True) , relu
		BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True) 
		Linear(in_features=50, out_features=3, bias=True) , none


In [7]:
Network_out_v_Label = np.hstack((Ŷ, data.Y))
wuml.jupyter_print(Network_out_v_Label)

,0,1,2,3,4,5
0,-0.141758,-0.159185,-1.581648,1.0,0.0,0.0
1,-0.191338,-0.309919,-1.604510,1.0,0.0,0.0
2,-0.584884,-0.231484,-2.200746,1.0,0.0,0.0
3,-1.368033,-0.561005,-2.408513,1.0,0.0,0.0
4,0.355720,-0.214580,0.260100,1.0,0.0,0.0
...,...,...,...,...,...,...
173,0.579378,-0.296554,-0.168530,0.0,0.0,1.0
174,0.554885,-0.298408,-0.105743,0.0,0.0,1.0
175,0.313081,-0.205449,-0.300636,0.0,0.0,1.0
176,0.318198,-0.194801,-0.342614,0.0,0.0,1.0
